<a href="https://colab.research.google.com/github/DreamPearl/Q-Learning-for-Trading/blob/master/Datascrib_Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Clone the entire repo.
%cd /content/
!git clone -l -s https://github.com/DreamPearl/Q-Learning-for-Trading.git Datascrib-Q-Learning
%cd Datascrib-Q-Learning
!ls

/content
fatal: destination path 'Datascrib-Q-Learning' already exists and is not an empty directory.
/content/Datascrib-Q-Learning
agent.py  envs.py   portfolio_val  README.md	     run.py    weights
data	  model.py  __pycache__    requirements.txt  utils.py


In [12]:
!git pull origin master

remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 3), reused 6 (delta 3), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/DreamPearl/Q-Learning-for-Trading
 * branch            master     -> FETCH_HEAD
   af00e80..7a619be  master     -> origin/master
Updating af00e80..7a619be
Fast-forward
 data/MSFT_Stock.csv | 1001 +++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 1001 insertions(+)
 create mode 100644 data/MSFT_Stock.csv


In [22]:
!python2.7 -m pip install gym==0.9.4

  Using cached https://files.pythonhosted.org/packages/f8/9f/b50f4c04a97e316ebfccae3104e5edbfe7bc1c687ee9ebeca6fa6343d197/gym-0.9.4.tar.gz
  Created wheel for gym: filename=gym-0.9.4-cp27-none-any.whl size=215482 sha256=cd33315dd3667b42c431c8c27798b7b7f9acae73e4973a61e34e8eba51fd1c5b
  Stored in directory: /root/.cache/pip/wheels/2f/ae/a0/126678adc5f1f8905309a4712ea29537999787f610edbcb0a4
Successfully built gym
ERROR: dopamine-rl 1.0.5 has requirement gym>=0.10.5, but you'll have gym 0.9.4 which is incompatible.
  Found existing installation: gym 0.10.11
    Uninstalling gym-0.10.11:
      Successfully uninstalled gym-0.10.11


In [26]:
!python2.7 run.py --mode train -e 1

Using TensorFlow backend.
[[0, 1428.0], [0, 100], [0, 40000]]
2020-12-03 10:53:11.887526: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-12-03 10:53:11.896566: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-03 10:53:11.896606: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2d0a1b5f9b89): /proc/driver/nvidia/version does not exist
2020-12-03 10:53:11.901695: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-12-03 10:53:11.901929: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557b6f90ebc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-12-03 10:53:11.901961: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Ver

In [29]:
!date
!ls -l weights/

Thu Dec  3 10:55:03 UTC 2020
total 236
-rw-r--r-- 1 root root 237600 Dec  3 10:53 202012031053-dqn.h5


In [30]:
!python2.7 run.py -m test -e 1 -w weights/202012031053-dqn.h5

Using TensorFlow backend.
[[0, 1428.0], [0, 100], [0, 40000]]
2020-12-03 10:56:16.068978: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-12-03 10:56:16.078206: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-03 10:56:16.078250: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2d0a1b5f9b89): /proc/driver/nvidia/version does not exist
2020-12-03 10:56:16.083487: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-12-03 10:56:16.083726: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x558175886bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-12-03 10:56:16.083760: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Ver

In [32]:
!git diff

diff --git a/agent.py b/agent.py
index 5091212..135f75d 100644
--- a/agent.py
+++ b/agent.py
@@ -51,9 +51,11 @@ class DQNAgent(object):
     target_f[range(batch_size), actions] = target
     
     history=self.model.fit(states, target_f,validation_split=0.2, epochs=1, verbose=0)
-    print('{},'.format(history.history['acc'][0]),end='')
+    # print(history.history)
+    # exit()
+    print('{},'.format(history.history['accuracy'][0]),end='')
     print('{},'.format(history.history['loss'][0]),end='')
-    print('{},'.format(history.history['val_acc'][0]),end='')
+    print('{},'.format(history.history['val_accuracy'][0]),end='')
     print(history.history['val_loss'][0],end='')
    
     if self.epsilon > self.epsilon_min:
diff --git a/envs.py b/envs.py
index 20036d3..9c89376 100644
--- a/envs.py
+++ b/envs.py
@@ -45,8 +45,11 @@ class TradingEnv(gym.Env):
     price_range = [[0, mx] for mx in stock_max_price]
     cash_in_hand_range = [[0, init_invest * 2]]
     rsi_range=[[0,100]]
+